<a href="https://colab.research.google.com/github/numarkevich/hse-homeworks/blob/master/%D0%93%D1%80%D0%B5%D0%B1%D0%BD%D0%B5%D0%B2%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F_%D0%B8_ANN_%D0%BD%D0%B0_%D0%BE%D1%87%D0%B8%D1%89%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import random
import math
import sklearn.datasets as ds
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16832262311840635653, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10971466682366195938
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

Found GPU at: 


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ml5_train.csv', sep=';')
data_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ml5_test.csv', sep=';')

In [0]:
data_train['age'] = data_train['age'].astype(int) #хочу, чтобы возраст стал более читаем
data_train['age'] = round(data_train['age'] *0.002739)

In [0]:
data_train.fillna(data_train.mean(), inplace=True)

In [0]:
data_train.gender -= 1

In [0]:
# Привожу в аналогичный вид тестовую выборку
data_test['age'] = data_test['age'].astype(int) #хочу, чтобы возраст стал более читаем
data_test['age'] = round(data_test['age'] *0.002739)
data_test.gender -= 1

In [0]:
data_train['height'].value_counts() # проверяю, где есть выбросы по каждому из параметров
# На основании своих находок чищу данные

165    5853
160    5022
170    4679
168    4399
164    3396
158    3313
162    3257
169    2791
156    2755
167    2538
163    2516
172    2016
159    1994
166    1979
157    1814
175    1809
155    1782
161    1712
174    1557
154    1443
171    1312
178    1227
152    1161
176    1145
173    1077
153    1059
150    1051
180     879
151     614
177     456
       ... 
75        2
104       2
131       1
66        1
128       1
71        1
72        1
96        1
200       1
74        1
60        1
59        1
250       1
122       1
57        1
119       1
55        1
113       1
112       1
111       1
108       1
99        1
98        1
97        1
91        1
81        1
80        1
207       1
76        1
64        1
Name: height, Length: 109, dtype: int64

In [0]:
data_train["height"].where(data_train["height"] < 200, inplace=True)
data_train["height"].where(data_train["height"] > 120, inplace=True)
data_train["age"].where(data_train["age"] < 100, inplace=True)
data_train["age"].where(data_train["age"] > 15, inplace=True)
data_train["weight"].where(data_train["weight"] < 250, inplace=True)
data_train["weight"].where(data_train["weight"] > 40, inplace=True)
data_train["ap_hi"].where(data_train["ap_hi"] < 300, inplace=True)
data_train["ap_hi"].where(data_train["ap_hi"] > 50, inplace=True)
data_train["ap_lo"].where(data_train["ap_lo"] < 200, inplace=True)
data_train["ap_lo"].where(data_train["ap_lo"] > 20, inplace=True)

In [0]:
data_train.fillna(data_train.mean(), inplace=True)

In [0]:
data_test.fillna(data_train.mean(), inplace=True)

In [0]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
id             70000 non-null int64
age            70000 non-null float64
gender         70000 non-null int64
height         70000 non-null float64
weight         70000 non-null float64
ap_hi          70000 non-null float64
ap_lo          70000 non-null float64
cholesterol    70000 non-null int64
gluc           70000 non-null int64
smoke          70000 non-null int64
alco           70000 non-null int64
active         70000 non-null int64
cardio         70000 non-null int64
dtypes: float64(5), int64(8)
memory usage: 6.9 MB


In [0]:
data_train["height"].astype(np.int64)

0        168
1        156
2        165
3        169
4        156
5        151
6        157
7        178
8        158
9        164
10       169
11       173
12       165
13       158
14       181
15       172
16       170
17       158
18       154
19       162
20       163
21       157
22       158
23       156
24       170
25       153
26       156
27       159
28       166
29       169
        ... 
69970    173
69971    177
69972    165
69973    175
69974    168
69975    182
69976    163
69977    168
69978    163
69979    163
69980    167
69981    182
69982    153
69983    165
69984    168
69985    156
69986    180
69987    151
69988    160
69989    157
69990    168
69991    159
69992    161
69993    172
69994    165
69995    168
69996    158
69997    183
69998    163
69999    170
Name: height, Length: 70000, dtype: int64

In [0]:
data_test["height"].astype(np.int64)
data_test['smoke'] = data_test.smoke.replace('None', -1).astype('int32')
data_test.active = data_test.active.replace('None', -1).astype('int32')
data_test.alco = data_test.alco.replace('None', -1).astype('int32')

In [0]:
# Гребневая регрессия
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
X, y = data_train.drop('cardio', axis=1), data_train['cardio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = Ridge(alpha=1.0)
clf.fit(X, y) 
ridge_clean = Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge_clean.fit(X, y) 
ridge_clean.score(X, y, sample_weight=None)

0.23004860548572803

In [0]:
predictions = ridge_clean.predict(X_test)
predictions

array([0.73278623, 0.89001912, 0.64648533, ..., 0.49288129, 0.54994146,
       1.03990787])

In [0]:
import keras

Using TensorFlow backend.


In [0]:
# ANN
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

import keras
from keras.models import Sequential
from keras.layers import Dense

# Инициализируем ANN
classifier = Sequential()

# Добавим входной слой и первый внутренний (скрытый) слой
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))

# Добавим второй внутренний слой
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))

# Добавим выходной слой
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Скомпилируем ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Обучим ANN на тренировочном датасете
classifier.fit(X_train, y_train, batch_size = 30, epochs = 50)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


Epoch 1/50
49000/49000 [==============================] - 3s 54us/step - loss: 0.5701 - acc: 0.7188
Epoch 2/50
49000/49000 [==============================] - 2s 36us/step - loss: 0.5498 - acc: 0.7289
Epoch 3/50
49000/49000 [==============================] - 2s 35us/step - loss: 0.5488 - acc: 0.7290
Epoch 4/50
49000/49000 [==============================] - 2s 35us/step - loss: 0.5480 - acc: 0.7305
Epoch 5/50
49000/49000 [==============================] - 2s 35us/step - loss: 0.5472 - acc: 0.7304
Epoch 6/50
49000/49000 [==============================] - 2s 36us/step - loss: 0.5467 - acc: 0.7295
Epoch 7/50
49000/49000 [==============================] - 2s 36us/step - loss: 0.5463 - acc: 0.7312
Epoch 8/50
49000/49000 [==============================] - 2s 35us/step - loss: 0.5457 - acc: 0.7307
Epoch 9/50
49000/49000 [==============================] - 2s 35us/step - loss: 0.5455 - acc: 0.7307
Epoch 10/50
49000/49000 [==============================] - 2s 36us/step - loss: 0.5452 - acc: 0.7307

In [0]:
# Сделаем предсказания и оценим нашу модель
from sklearn.metrics import roc_auc_score
y_pred = classifier.predict_proba(X_test)
roc_auc_score(y_test, y_pred)

0.8010455364196

In [0]:
y_pred

array([[0.8203265 ],
       [0.8494463 ],
       [0.6390582 ],
       ...,
       [0.7119899 ],
       [0.58878475],
       [0.88836896]], dtype=float32)